In [1]:
from Bio import SeqIO

D = []
with open("NEWSetD_50seq_peptide.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        D.append(str(record.seq))


In [ ]:
import numpy as np

def local_alignment(seq1, seq2, match=2, mismatch=-1, gap=-1):

    m, n = len(seq1), len(seq2)
    score_matrix = np.zeros((m + 1, n + 1), dtype=int)
    
    max_score = 0
    max_pos = None
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match_score = score_matrix[i -1, j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            delete = score_matrix[i-1, j] + gap
            insert = score_matrix[i, j-1] + gap
            score_matrix[i, j] = max(0, match_score, delete, insert)
            if score_matrix[i, j]> max_score:
                max_score = score_matrix[i, j]
                max_pos = (i, j)
    return score_matrix, max_score, max_pos

def traceback(score_matrix, seq1, seq2, max_pos):
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_pos
    while score_matrix[i, j] != 0:
        if score_matrix[i, j] == score_matrix[i - 1, j - 1] + (2 if seq1[i - 1] == seq2[j - 1] else -1):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif score_matrix[i, j] == score_matrix[i - 1, j] - 1:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
    return ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))

def calculate_percent_identity(aligned_seq1, aligned_seq2):
    matches = sum(1 for a, b in zip(aligned_seq1, aligned_seq2) if a == b)
    length = len(aligned_seq1)
    return (matches/length) * 100

# seq1 = D[0]
# seq2 = D[1]
# score_matrix, max_score, max_pos = local_alignment(seq1, seq2)
# aligned_seq1, aligned_seq2 = traceback(score_matrix, seq1, seq2, max_pos)
# percent_identity = calculate_percent_identity(aligned_seq1, aligned_seq2)

# print("Local alignment matrix:")
# print(score_matrix)
# print("Max score:", max_score)
# print("Max position:", max_pos)
# print("Aligned sequences:")
# print(aligned_seq1)
# print(aligned_seq2)
# print("Percent identity:", percent_identity)

Local alignment matrix:
[[  0   0   0 ...   0   0   0]
 [  0   2   1 ...   0   0   0]
 [  0   1   1 ...   0   2   1]
 ...
 [  0   0   0 ... 275 274 273]
 [  0   0   0 ... 274 277 276]
 [  0   0   0 ... 273 276 276]]
Max score: 277
Max position: (322, 282)
Aligned sequences:
AA-SPYERGPAPTVASIEA--A-RGPFAVAQTTVSRFAASGFGGGTITYPTSTAEGTFGAVAISPGYTAAQSSISWLGPRLASQGFVVITIDTN--SRYDQPSSRGDQLRAALAYLTGSSSVRGRIDASRLAVMGHSMGGGGALEAAKDNPALQAAIPMTPWNTDKTWPE--VQTPTLIIGAQNDSVASVTTHAEPFYEN-LPGSLDKAYLELRGASHFAP-NTSNT-TIARQSIAWLKRFVDDDERYAQFLCPPPSGTAISEYRSTCP
AAENPYERGPAPT-LS--ALQASRGPYAVSQTSVSNLAATGFGGGEIYYPTTTADGTFGAIAISPGFTAYWSSISWLGPRLASHGFVVIGIETNTTS--DQPASRGDQLLAALDYLTQRSSVRNRVDASRLAVGGHSMGGGGSLEAASDRPSLQAAIPLAPWNLDKSWSELRV--PTLIIGGESDSIASVSSHSIPFY-NSIPASAEKSYLELNNASHFFPQST-NTPT-AVQAVAWLKRWVDNDTRYSQFICPGPSSTSISDYRSSCP
Percent identity: 67.65799256505576


In [8]:
M1 = "DSIAPVNSSALPIYDSMSRNAKQFLEINGGSH"

In [ ]:
results = []

for idx, seq in enumerate(D):
    score_matrix, max_score, max_pos = local_alignment(M1, seq)
    aligned_seq1, aligned_seq2 = traceback(score_matrix, M1, seq, max_pos)
    percent_identity = calculate_percent_identity(aligned_seq1, aligned_seq2)
    if percent_identity > 50:
        results.append({
            'seq_num': idx,
            'seq': seq,
            'aligned_seq1': aligned_seq1,
            'aligned_seq2': aligned_seq2,
            'percent_identity': percent_identity
        })
for result in results:
    print("Sequence Number:",result['seq_num'])
    print("Sequence:", result['seq'])
    print("Aligned Sequence 1:",result['aligned_seq1'])
    print("Aligned Sequence 2:", result['aligned_seq2'])
    print("Percent Identity:", result['percent_identity'])
    print()

Sequence Number: 0
Sequence: MPLVATLGLALAAGSLAPSASAAAGDPATVETPPVVSQRAIDEHAATEPQLSLDAPAALPPVTIAASPYERGPAPTVASIEAARGPFAVAQTTVSRFAASGFGGGTITYPTSTAEGTFGAVAISPGYTAAQSSISWLGPRLASQGFVVITIDTNSRYDQPSSRGDQLRAALAYLTGSSSVRGRIDASRLAVMGHSMGGGGALEAAKDNPALQAAIPMTPWNTDKTWPEVQTPTLIIGAQNDSVASVTTHAEPFYENLPGSLDKAYLELRGASHFAPNTSNTTIARQSIAWLKRFVDDDERYAQFLCPPPSGTAISEYRSTCPY
Aligned Sequence 1: SIAPVNS-SA
Aligned Sequence 2: SLAP--SASA
Percent Identity: 60.0

Sequence Number: 2
Sequence: MSAARPGGPPRPRGRLVVQQHPRSTTASAAPGPARGAGRRGTRRFAGAAAAIAAAVALSTLTGPGARAADNPYERGPAPTTASIEASRGPYSVSETSVSSLAVSGFGGGTIYYPTSTADGTFGAVAVSPGYTGTQSSIAWLGPRLASQGFVVFTIDTLTTLDQPDSRGRQLLAALDYLTTRSSVRGRVDSTRLGVMGHSMGGGGSLEAAKSRPSLQAAIPLTPWNLDKSWPEVTTPTLIVGADGDSIAPVSSHAEPFYGSLRSSLDRAYLELNGASHFTPNSSNTTIAKYSVSWLKRFIDNDTRYEQFLCPLPSPSLTIEEYRGNCPHTS
Aligned Sequence 1: DSIAPVNSS-ALPIYDS--MS--RNAKQFLEINGGSH
Aligned Sequence 2: DSIAPV-SSHAEPFYGSLRSSLDR-A--YLELNGASH
Percent Identity: 56.75675675675676

Sequence Number: 4
Sequence: MSSPTTTVRPRPAVRLAGL